# Reality: most of your dark frame is noise and not all of the time dependent artifacts are dark current

The dark current on modern CCDs, even relatively inexpensive ones, is very small. In a typical exposure the dark current is likely smaller, maybe much smaller, than the read noise. Despite that it is worth examining some dark frames from your camera. because not all pixels have the same dark current. Some, called hot pixels, have much higher dark current than the rest of the sensor.

As we did in the [notebook about overscan](01.08-Overscan.ipynb), we will focus on a pair of cameras to illustrate this point and demonstrate how to take similar images with your camera and analyze them.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u
from astropy.nddata import CCDData
from astropy.visualization import hist
from astropy.stats import mad_std
from ccdproc import ImageFileCollection, subtract_overscan, trim_image, combine, subtract_bias

from convenience_functions import show_image

## Case 1: Cryogenically cooled  Large Format Camera (LFC) at Palomar

The images in this section are from chip 0 of the LFC at the Palomar 200-inch telescope. Technical information about the camera is [here](http://www.astro.caltech.edu/palomar/observer/200inchResources/lfcspecs.html). The technical information says nothing about the expected dark current of this camera. The expectation for cameras cooled by liquid nitrogen is that their dark current is essentially zero.

What we will see with this camera is that

+ not all of the pixels have negligible dark current, and 
+ not all of the counts in a dark image are from dark current.



In [ ]:
darks = ImageFileCollection(Path('python_imred_data') / 'darks')
darks.summary['file', 'exptime']

### Best we can do is 3x 300 sec

Let's grab the calibrated bias...

In [ ]:
calib_bias = CCDData.read(Path('.') / 'reduced' / 'combined_bias.fit')
calibrated_images = Path('.') / 'reduced'

In [ ]:
d_to_combine = []

for d in darks.ccds(exptime=300.0, ccd_kwargs=dict(unit='adu')):
    new_d = subtract_overscan(d, overscan=d[:, 2055:], median=True)
    new_d = trim_image(new_d[:, :2048])
    new_d = subtract_bias(new_d, calib_bias)
    d_to_combine.append(new_d.copy())

In [ ]:
d_to_combine

In [ ]:
d_to_combine[0].dtype

In [ ]:
%%time
combined_dark = combine(d_to_combine, output_file=calibrated_images / 'combined_300s_darks.fits',
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                             mem_limit=350e6
                            )

In [ ]:
combined_dark_lfc = CCDData.read(calibrated_images / 'combined_300s_darks.fits')

In [ ]:
show_image(combined_dark, cmap='gray')

In [ ]:
show_image(combined_dark[:200, :200], cmap='gray')

In [ ]:
gain = 2.0
exposure_time = 300
dark_current_ln2 = gain * combined_dark.data / exposure_time

plt.figure(figsize=(20, 10))
hist(dark_current_ln2.flatten(), bins=5000, density=False, alpha=0.5, color='C0');
hist(dark_current_ln2[200:, 200:].flatten(), bins=5000, density=False, alpha=0.5, color='C1')
hist(dark_current_ln2[:200, :200].flatten(), bins=5000, density=False, alpha=0.5, color='C9');
#plt.semilogy()
plt.loglog()

plt.grid()
plt.vlines(2 * 11/exposure_time/np.sqrt(3), 0.1, 8e6)
plt.ylim(0.1, 8e6)
#plt.xlim(0, .1)
plt.xlabel('dark current, electrons per second')
plt.ylabel('Number of pixels with that dark current');


## Case 2: Thermo-electrically cooled camera

### Making an image to check for hot pixels

The image below is a combination of 20 dark frames, each a 1000 sec exposure. That exposure time was chosen so that the expected number of dark electrons was at least somewhat larger than the read noise expected in a combination of 20 images. As we saw in the previous notebook, unless that is the case the dark frames will measure read noise instead of dark current.

This camera has a dark current of $0.01 e^-$/sec/pixel and read noise of roughly $10 e^-$/read/pixel/. An exposure time of 1000 sec means an expected dark level in each exposure of $10 e^-$. By combining 20 of the images, the expected noise in the combination is reduced by a factor of $\sqrt{20}$ to $2.2 e^-$/pixel. This puts us in the "low read noise" limit in the previous notebook about ideal dark current.

Measuring the dark current once you have the images is relatively straightforward: subtract the bias, multiply the result by the gain and then divide by the exposure time to obtain the dark current in $e^-$/pixel/sec.

In [ ]:
dark_1000 = CCDData.read('https://zenodo.org/record/2634177/files/master_dark_exposure_1000.0.fit.bz2?download=1')
show_image(dark_1000, cmap='gray')

### Calculate the dark current for each pixel

Recall that the dark current $d_c(T)$ is given by 

$$
d_c(T) = d_e(t) / t,
$$

where $d_e(t)$ is the number of dark electrons in the images. That is related to the dark counts, $n_{dark}(t)$, the image values displayed in the image above, by the gain of the camera, $d_e(t) = g n_{dark}(t)$, so that

$$
d_C(T) = g n_{dark}(t) / t.
$$

This particular camera has a gain of $g = 1.5 e^-$/ADU.

In [ ]:
gain = 1.5
exposure_time = 1000
dark_current = gain * dark_1000.data / exposure_time

plt.figure(figsize=(20, 10))
hist(dark_current.flatten(), bins=5000, density=False);
plt.vlines(10/exposure_time/np.sqrt(20), *plt.ylim())
#plt.semilogy()
plt.loglog()
plt.grid()
#plt.xlim(0, .1)
plt.xlabel('dark current, electrons per second')
plt.ylabel('Number of pixels with that dark current');


### There is a large range in the dark current 

While the vast majority of the pixels do, as expected, have very low dark current, it is much higher for other pixels. These pixels, called hot pixels, can occur even in cryogenically-cooled cameras. 

The vast majority of the pixels in the image have a dark current around the value promised by the manufacturer, 0.01 $e^-$/sec; the highest nominally have dark current 98 $e^-$/sec.

However, there is  an upper limit to the dark current that can be measured in a particular exposure because there is an upper limit to the number of counts a CCD can represent. For this camera that limit is $2^{16} -1$, or $65,563$. If we convert that to a dark current for a 1000 sec exposure, it is approximately 98 $e^-$/sec. For the hottest pixels in this image, that is the *lower limit* of the dark current for those pixels.

The dark current is also not well estimated for pixels a bit below the maximum because CCDs stop responding linearly once they pass a pixel value that depends on the camera. For this camera, the linearity limit is around 55,000 counts, corresponding to a dark current of roughly  82 $e^-$/sec.

One can, in principle, also check whether the hot pixels have a constant dark current that does not change with time by creating a number of these darks at different exposure times and plotting the dark current as a function of exposure time for the hot pixels. If the dark current is constant then the dark counts will be properly removed when the dark is subtracted from the science image. If the dark current is not constant, the pixel should be excluded from analysis.

The fraction of very hot pixels, those above 1 $e^-$/sec, is relatively small, roughly 0.1% of the pixels.

In [ ]:
plt.figure(figsize=(20, 10))
hist(dark_current_ln2[200:, 200:].flatten(), bins=5000, density=True, alpha=0.5, label='Cryogenic, excluding sensor glow');
hist(dark_current.flatten(), bins=5000, density=True, alpha=0.5, label='Thermo-electric');
plt.grid()
plt.loglog()
plt.xlabel('dark current, electrons per second')
plt.ylabel('Number of pixels with that dark current');
plt.legend();

## How to handle hot pixels

There are a few ways one could handle this, which can be used in combination with each other:

+ Mark all pixels above some threshold as bad and create a mask to keep track of these bad pixels.
+ Mark only the really high dark current pixels as bad and mask them; for the rest, subtract dark current as usual.
+ Always take dark images with exposure times that match your flat and light frames and subtract dark current as usual.
+ Take one set of darks with exposure equal to the *longest* exposure time in your flat and light images and scale the dark current downwards to match your other exposure times.

## One thing NOT to do

Do not take short dark frames and scale them up to longer exposure times. Modern cameras, even thermo-electrically cooled ones, have very low dark current. If your dark frames have low exposure time then most of the pixels are measuring read noise, not dark current. If you rescale those images to a longer exposure time then you inappropriately amplify that noise. Ideally, the expected dark counts (or dark electrons) in your dark frames should be at least a few times larger than the expected read noise in the frames you combine to make a reference dark.

## Summary

1. A dark frame only measures dark current if the expected dark counts exceed the read noise of the camera by a factor of a few.
2. Take multiple dark frames and combine them to reduce the noise level in the combined image as much as possible.
2. Most pixels in a CCD have very low dark current.
3. A consequence of 1 and 2 is that you should almost never scale your dark frames up to a longer exposure time because you will amplify noise instead of eliminating dark current.
3. Identify hot pixels and mask them out or otherwise deal with them.
